In [287]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [309]:
def data_ret():
    dataset = pd.read_csv('E:\ELL_project\problem1\health_data.csv')
    dataset = dataset.sample(frac = 1)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1:].values
    datasize = X.shape[0]
    X_train = X[:(datasize*7)//10,:]
    y_train = y[:(datasize*7)//10,:]
    X_test = X[(datasize*7)//10:,:]
    y_test = y[(datasize*7)//10:,:]

    X_train_mean = np.sum(X_train,axis=0).reshape(1,-1)/X_train.shape[0]
    X_train_var = np.sqrt(np.sum(np.square(X_train-X_train_mean),axis=0).reshape(1,-1)) / X_train.shape[0]
    X_train = (X_train-X_train_mean) / X_train_var
    X_train = np.append(np.ones((X_train.shape[0],1)),X_train,axis=1)
    X_test = (X_test-X_train_mean) / X_train_var
    X_test = np.append(np.ones((X_test.shape[0],1)),X_test,axis=1)
    return X_train,y_train,X_test,y_test

In [106]:
# check

In [289]:
def gradDesc(X,y,theta,hypothesis,loss_function,regularizer,alpha,lambda1,lambda2):
    h,loss,gradient = loss_function(X,theta,y,hypothesis)
    if(regularizer==elastic_net_reg):
        reg_loss,reg_grad = regularizer(lambda1,lambda2,theta)
    else:
        reg_loss,reg_grad = regularizer(lambda1,theta)
    # loss += reg_loss
    gradient += reg_grad

    return(loss,gradient)

In [290]:
def mse_loss(X,theta,y,hypothesis):
    siz = y.shape[0]
    h = hypothesis(X,theta)
    diff = h-y
    mse = ( np.sum(np.square(diff),axis=0))[0]
    gradient = np.dot(np.transpose(X),diff) / siz
    return(h,mse,gradient)

def mae_loss(X,theta,y,hypothesis):
    siz = y.shape[0]
    num_feat = y.shape[1]
    h = hypothesis(X,theta)
    diff = h-y
    diff_sign = np.ones((siz,1))
    diff_sign[diff[:,0]<0] = -1
    mae = np.sum(np.abs(h),axis=0)[0] / siz
    gradient = np.sum(diff_sign*X,axis=0).reshape(-1,1)
    return(h,mae,gradient)

def ce_loss(X,theta,y,hypothesis):
    siz = y.shape[0]
    h = hypothesis(X,theta)
    h_comp = 1-h
    diff = h - y
    ce = np.sum(-y*np.log(h)-(1-y)*np.log(1-h),axis=0) [0]
    gradient = np.dot(np.transpose(X),diff).reshape(-1,1) / siz
    return(h,ce,gradient)   

In [291]:
def lin_hyp(X,theta):
    # print(np.dot(X,theta).shape)
    return (np.dot(X,theta))
def log_hyp(X,theta):
    a = lin_hyp(X,theta)
    return ( 1/(1+np.exp(-a)) )
def null_regularizer(alpha,theta):
    a = np.zeros(theta.shape)
    return(0,a)
def l1_reg(alpha,theta):
    reg_loss = alpha*theta
    reg_grad = alpha
    return(reg_loss,reg_grad)
def l2_reg(alpha,theta):
    reg_loss = alpha * np.square(theta)
    reg_grad = 2 * alpha * theta
    return(reg_loss,reg_grad)
def elastic_net_reg(lambda1,lambda2,theta):
    a1,a2 = l1_reg(lambda1,theta)
    b1,b2 = l2_reg(lambda2,theta)
    return( a1+b1 , a2+b2 )

In [301]:
def linReg(X,y,iter=1000,alpha=0.01,batchSize=32,hypothesis = lin_hyp,loss_function=mse_loss,regularizer=l2_reg,lambda1 = 0.01,lambd2_elasnet = 0.01):
    theta = np.random.random((X.shape[1],1))
    # print(theta)
    # print(theta)
    datasize = X.shape[0]
    loss_epoch = 0
    for i in range(iter):
        if((i+1)%100==0 and i>0):
            print('Loss for {} iterations: {}'.format(i+1,loss_epoch))
        fro = 0
        loss_epoch = 0
        while(True):
            to = min(fro+batchSize,datasize)
            l,theta_grad = gradDesc(X[fro:to,:],y[fro:to,:],theta,hypothesis,loss_function,regularizer,alpha,lambda1, lambd2_elasnet)
            # print(theta_grad)
            loss_epoch += l
            # print(theta_grad)
            theta -= (alpha*theta_grad)
            fro = to

            if(to>=datasize):
                break
    
    return (theta,loss_epoch)


In [311]:
X_train,y_train,X_test,y_test = data_ret()
train_datasize = X_train.shape[0]
opt_theta, train_loss = linReg(X_train,y_train,iter=1000,alpha=0.001,batchSize=5,hypothesis = lin_hyp,loss_function=mse_loss,regularizer=l1_reg,lambda1=0.01,lambd2_elasnet = 0.001)
print(opt_theta)

Loss for 100 iterations: 52.49571111884227
Loss for 200 iterations: 52.49575782710468
Loss for 300 iterations: 52.495757829391806
Loss for 400 iterations: 52.495757829391934
Loss for 500 iterations: 52.495757829391934
Loss for 600 iterations: 52.495757829391934
Loss for 700 iterations: 52.495757829391934
Loss for 800 iterations: 52.495757829391934
Loss for 900 iterations: 52.495757829391934
Loss for 1000 iterations: 52.495757829391934
[[ 4.10957068e-01]
 [ 1.53491848e-02]
 [ 7.15529401e-03]
 [-3.36786273e-04]]


In [308]:
y_pred = np.dot(X_test,opt_theta)
loss_y = y_pred - y_test

test_size = y_pred.shape[0]
total_loss_y = np.dot(np.ones((1,test_size)),np.square(loss_y))[0,0] / test_size

y_pred_thresh = y_pred>=0.5

tp = np.sum((y_pred_thresh+y_test)==2 , axis=0)[0]
tn = np.sum(y_pred_thresh==y_test , axis=0)[0] - tp
fp = np.sum(y_pred_thresh , axis=0)[0]-tp
fn = test_size-tp-tn-fp

print('tp: {} , tn: {} , fp: {} , fn: {}'.format(tp,tn,fp,fn))

acc = (tp+tn)/test_size
prec = (tp)/(tp+fp)
recl = (tp)/(tp+fn)
f1 = 2*prec*recl/(prec+recl)
print('Accuracy: {}'.format( acc  ))
print('Precision: {}'.format( prec  ))
print('Recall: {}'.format( recl  ))
print('F1 score: {}'.format( f1  ))

tp: 0 , tn: 134 , fp: 0 , fn: 76
Accuracy: 0.638095238095238
Precision: nan
Recall: 0.0
F1 score: nan


In [214]:
# import matplotlib.pyplot as plt
# import numpy as np

# x = np.array([1,2,3,4,5])
# y = np.array([2,1,3,6,7])

# cluster = np.array([1,1,1,2,2]) 

# fig, ax = plt.subplots()

# ax.scatter(x[cluster==1],y[cluster==1], marker='^')
# ax.scatter(x[cluster==2],y[cluster==2], marker='s')

# plt.show()